# Próximo HIT Spotify

## Importando as bibliotecas

Aqui é importado as bibliotecas necessárias para a executação do notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV

## Carregando o Dataset
Carrega o CSV para criar um dataframe.

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

## Análise e Exploração dos dados

Nesta etapa, serão realizadas as seguintes atividades:
- **Informações gerais**: Coleta e apresentação de informações básicas sobre o conjunto de dados.
- **Identificação das colunas numéricas e categóricas**: Classificação das variáveis do conjunto de dados em numéricas e categóricas, além das principais características das variáveis.

In [ ]:
train_data.head()

In [ ]:
test_data.head()

## Visualização e exploração de dados

Aqui, será abordado as técnicas utilizados para a visualização e exploração inicial dos dados. Através disso, é possível obter uma compreensão prévia da estrutura e das características do conjunto de dados.

* As colunas do DataFrame são dividias em categóricas e numéricas para análise e tratamento.
* Então primeiro é preciso identificar as colunas categóricas e numéricas, isso será feito nas primeira linha do código e por fim nas últimas duas colunas exibiremos a quantidade e os nomes das colunas.

Com o método `.info()` é possível obter informações detalhadas sobre o DataFrame. Especificamente, ele permite analisar os tipos de variáveis presentes no conjunto de dados.

Utilizando o comando `shape` abaixo, se observa que há 79800 linhas e 21 colunas a serem tratadas.

In [ ]:
train_data.info()
train_data.shape

In [ ]:
test_data.info()
test_data.shape

O método `.describe()` é utilizado para fornecer um resumo estatísticodas variáveis numéricas. Ao utilizar este comando, obtemos informações como a média, desvio padrão, valores mínimo e máximo, além dos quartis. Com esses dados é possível entender a distribuição e a variabilidade dos dados, ajudando a identificar padrões, valores nulos e *outliers*.

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

## Valores Nulos

É feita a verificação de valores nulos no DataFrame utilizando o método `isnull().sum()`. Ele permite contar a quantidade de valores NaN em cada coluna, facilitando a identificação de colunas que necessitam de tratamento para lidar com dados ausentes.


In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

Após analisar que posssuia linhas com valores nulos, é feita a remoção delas.

In [11]:
test_data.fillna(0, inplace=True)

In [ ]:
# Identifica linhas duplicadas
train_data[train_data.duplicated(keep='first')]

In [ ]:
# Identifica linhas duplicadas
test_data[test_data.duplicated(keep='first')]

## Identificação e Seleção de Colunas

É realizado a identificação das colunas presentes no dataset, classificando-as em numéricas e categóricas.

In [ ]:
#Coloca em uma variável todas as colunas que tem dados do tipo categórico
colunas_categoricas = train_data.select_dtypes(include='object').columns
#Coloca em uma variável todas as colunas que train_data do tipo numérico
colunas_numericas = train_data.drop(colunas_categoricas, axis=1).columns

# Imprime a lista de colunas categóricas
print(f'Há {len(colunas_categoricas)} Colunas Categóricas: {list(colunas_categoricas)}')

# Imprime a lista de colunas numéricas
print(f'Há {len(colunas_numericas)} Colunas Numéricas: {list(colunas_numericas)}')

In [ ]:
#Coloca em uma variável todas as colunas que tem dados do tipo categórico
colunas_categoricas = test_data.select_dtypes(include='object').columns
#Coloca em uma variável todas as colunas que test_data do tipo numérico
colunas_numericas = test_data.drop(colunas_categoricas, axis=1).columns

# Imprime a lista de colunas categóricas
print(f'Há {len(colunas_categoricas)} Colunas Categóricas: {list(colunas_categoricas)}')

# Imprime a lista de colunas numéricas
print(f'Há {len(colunas_numericas)} Colunas Numéricas: {list(colunas_numericas)}')

## Análise de Gráficos

### Gráfico 1: Distribuição de Popularidade

Neste gráfico é possível observar a distribuição da popularidade entre as músicas.

In [ ]:
# Configura o tamanho padrão dos gráficos
plt.figure(figsize=(12, 6))

# Analisa a distribuição da variável alvo (popularity_target)
plt.figure(figsize=(8, 8))
popularity_counts = train_data['popularity_target'].value_counts()
plt.pie(popularity_counts, labels=['Não Popular', 'Popular'], autopct='%1.1f%%', startangle=90, colors=['#66b312','#9931f9'])
plt.title('Proporção de Músicas Populares e Não Populares')
plt.show()


### Gráfico 2: Distribuição da Energia das Músicas
Mostra a variação da energia das músicas, com a maioria delas se concentrando entre valores médios e altos de energia.

In [ ]:
# Distribuição de 'energy' (energia) para ver como essa variável está distribuída
plt.figure(figsize=(10, 5))
sns.histplot(train_data['energy'], bins=20, kde=True)
plt.title('Distribuição da Energia das Músicas')
plt.xlabel('Energia') 
plt.ylabel('Número de músicas') # Quantidade de músicas
plt.show()


### Gráfico 3: Variação entre Danceabilidade e Valência
Mostra a relação entre a "dançabilidade" e a "valência" (emocionalidade positiva) das músicas.

In [ ]:
# Gráfico de dispersão entre 'danceability' (dançabilidade) e 'valence' (positividade emocional)
plt.figure(figsize=(10, 5))
sns.scatterplot(x='danceability', y='valence', data=train_data, color='darkblue')
plt.title('Dançabilidade vs Valência')
plt.xlabel('Dançabilidade') # Dançabilidade
plt.ylabel('Positividade Emocional') # Positividade emocional
plt.show()


## Identificação de Outliers

Nessa seção foi feita a identificação e tratamento de outliers. Outliers são valores que se distanciam significativamente do restante dos dados, podendo distorcer análises e influenciar negativamente o desempenho do modelos preditivos.

In [ ]:
# Função para identificar outliers
for column in train_data.select_dtypes(include=['number']).columns:
    def identificar_outliers_iqr(coluna):
      Q1 = coluna.quantile(0.25)
      Q3 = coluna.quantile(0.75)
      IQR = Q3 - Q1
      limite_inferior = Q1 - 1.5 * IQR
      limite_superior = Q3 + 1.5 * IQR
      return (coluna < limite_inferior) | (coluna > limite_superior)

# Aplica a função apenas nas colunas numéricas
outliers = train_data.select_dtypes(include=[np.number]).apply(identificar_outliers_iqr)

# Conta a quantidade de outliers por coluna
quantidade_outliers = outliers.sum()

# Imprime o resultado
print(quantidade_outliers)

Aqui foi feito a visualização através de boxplots para identificar outliers nas colunas numéricas.

In [ ]:
# Função para plotar boxplots com outliers
def plot_boxplots_with_outliers(dados, max_plots_per_fig=10):
    # Filtra colunas numéricas dos dados
    numeric_cols = dados.select_dtypes(include=['number']).columns
    # Número de figuras necessárias pra plotar as colunas
    num_figures = int(np.ceil(len(numeric_cols) / max_plots_per_fig))
    for fig_idx in range(num_figures):
        start_idx = fig_idx * max_plots_per_fig
        end_idx = start_idx + max_plots_per_fig
        cols_to_plot = numeric_cols[start_idx:end_idx]
        fig, axes = plt.subplots(nrows=len(cols_to_plot), ncols=1, figsize=(8, len(cols_to_plot) * 4))
        
        # Se tiver só uma coluna, não gera a lista
        if len(cols_to_plot) == 1:
            axes = [axes]
        for ax, col in zip(axes, cols_to_plot):
            try:
                sns.boxplot(x=dados[col], ax=ax)
                ax.set_title(f'Boxplot de {col}')
                ax.set_xlabel(col)
            except ValueError as e:
                print(f"Erro ao criar boxplot para a coluna '{col}': {e}")
        plt.tight_layout()
        plt.show()
        
# Informações de diagnóstico
print(train_data.info())
print(train_data.head())

# Plota os boxplots
plot_boxplots_with_outliers(train_data)


O tratamento foi feito baseado na utilização de Interquartir(IQR) para a remoção dos outliers.

In [ ]:
# Função para remover outliers usando IQR
def remover_outliers_iqr(dados):
    for coluna in dados.select_dtypes(include=[np.number]).columns:
        Q1 = dados[coluna].quantile(0.25)
        Q3 = dados[coluna].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        # Mantém apenas os dados que estão dentro dos limites
        dados = dados[(dados[coluna] >= limite_inferior) & (dados[coluna] <= limite_superior)]
    return dados

# Remove outliers do conjunto de dados
train_data = remover_outliers_iqr(train_data)

# Verifica o tamanho do novo conjunto de dados após a remoção dos outliers
train_data.shape


## Hipóteses

### Hipótese 1: Músicas com maior nível de energia tendem a ser mais populares.
Músicas com alta energia costumam ser mais animadas, características frequentemente associadas a músicas populares, especialmente em gêneros como pop e eletrônico.

In [ ]:
# Calcula a média de energia para cada classe de popularidade
energy_mean_by_popularity = train_data.groupby('popularity_target')['energy'].mean()

# Cria gráfico de barras para a média de energia
plt.figure(figsize=(8, 5))
energy_mean_by_popularity.plot(kind='bar', color=['#66b3ff', '#99ff99'])
plt.title('Média de Energia para Músicas Populares e Não Populares')
plt.xlabel('Popularidade (0 = Não Popular, 1 = Popular)')
plt.ylabel('Média de Energia')
plt.xticks(rotation=0)
plt.show()


#### O gráfico, ao contrário do que pensava, mostra que não depende somente de energia para ser popular.

### Hipótese 2: Músicas com alta "danceability" (dançabilidade) têm maior chance de serem populares.
Músicas dançantes são frequentemente tocadas em festas e eventos, o que pode impulsionar sua popularidade no Spotify, especialmente entre jovens.

In [ ]:
# Calcula a média de danceabilidade para cada classe de popularidade
danceability_mean_by_popularity = train_data.groupby('popularity_target')['danceability'].mean()

# Cria gráfico de barras para a média de danceabilidade
plt.figure(figsize=(8, 5))
danceability_mean_by_popularity.plot(kind='bar', color=['#884111', '#132309'])
plt.title('Média de Danceabilidade para Músicas Populares e Não Populares')
plt.xlabel('Popularidade (0 = Não Popular, 1 = Popular)')
plt.ylabel('Média de Danceabilidade')
plt.xticks(rotation=0)

plt.show()


#### É possível entender dessa hipótese que existe um padrão de média da dançabilidade entre as músicas não populares e populares.

### Hipótese 3: Músicas de gêneros específicos, como pop e eletrônico, são mais propensas a serem populares do que músicas de gêneros de nicho, como jazz ou clássica.
Gêneros populares tendem a ter maior audiência em plataformas de streaming como o Spotify, o que aumenta a probabilidade de uma música nesses gêneros alcançar um público maior.

In [ ]:
# Limita o gráfico para mostrar apenas os 15 gêneros mais populares
track_genre = train_data.groupby('track_genre')['popularity_target'].mean().sort_values(ascending=False).head(15)

# Cria o gráfico de barras com os gêneros mais populares
plt.figure(figsize=(10, 5))
track_genre.plot(kind='barh', color='blue')
plt.title('Top 15 Gêneros Musicais com Maior Popularidade Média')
plt.xlabel('Gênero Musical')
plt.ylabel('Média de Popularidade')
plt.xticks(rotation=0)
plt.gca().invert_yaxis()  # Inverte o eixo y para que o gênero mais importante apareça no topo
plt.show()

#### A partir do gráfico, conseguimos observar que gêneros musicais que são mais regionais, como forró e sertanejo (populares no interior do Brasil), além de gêneros específicos de determinadas regiões do mundo, como música indiana e turca, tendem a ter maior popularidade em suas respectivas áreas.

## Codificação de variáveis categóricas

Aqui é feito a codificação da variável `track_name` e `track_genre` para o formato numérico, permitindo o modelo interpretar os nomes das músicas como entradas válidas.

In [ ]:
# Inicializa o LabelEncoder
label_encoder = LabelEncoder()

# Codifica os nomes das músicas
train_data['track_name_encoded'] = label_encoder.fit_transform(train_data['track_name'])

# Exclui as colunas originais dos nomes das músicas
train_data = train_data.drop(columns=['track_name'])

# Ordena as colunas codificadas em ordem crescente
train_data = train_data.sort_values(by=['track_name_encoded']).reset_index(drop=True)

train_data.head(10)

In [ ]:
# Inicializa o LabelEncoder
label_encoder = LabelEncoder()

# Codifica a coluna 'genre'
label_encoder = LabelEncoder()
train_data['genre_encoded'] = label_encoder.fit_transform(train_data['track_genre'])

# Exclui as colunas originais dos gêneros das músicas
train_data = train_data.drop(columns=['track_genre'])

# Ordena as colunas codificadas em ordem crescente
train_data = train_data.sort_values(by=['genre_encoded']).reset_index(drop=True)

train_data.head(10)

### Codificação das variáveis categóricas `test.csv`

In [ ]:
# Inicializa o LabelEncoder
label_encoder = LabelEncoder()

# Converte todos os valores de 'track_name' para string
test_data['track_name'] = test_data['track_name'].astype(str)

# Codifica os nomes das músicas
test_data['track_name_encoded'] = label_encoder.fit_transform(test_data['track_name'])

# Exclui as colunas originais dos nomes das músicas
test_data = test_data.drop(columns=['track_name'])

# Ordena as colunas codificadas em ordem crescente
test_data = test_data.sort_values(by=['track_name_encoded']).reset_index(drop=True)

test_data.head(10)

In [ ]:
# Inicializa o LabelEncoder
label_encoder = LabelEncoder()

# Codifica a coluna 'genre'
label_encoder = LabelEncoder()
test_data['genre_encoded'] = label_encoder.fit_transform(test_data['track_genre'])

# Exclui as colunas originais dos gêneros das músicas
test_data = test_data.drop(columns=['track_genre'])

# Ordena as colunas codificadas em ordem crescente
test_data = test_data.sort_values(by=['genre_encoded']).reset_index(drop=True)

test_data.head(10)

## Treinamento do Modelo

Para o treinamento, foi escolhido o algoritmo Random Forest, para prever se uma música será popular no Spotify, com base em suas características acústicas.

Foram feitos algumas ações durante esse processo, assim como:

- Imputação: Valores ausentes foram substituídos pela média das colunas.

- Padronização: As features foram escaladas para manter a mesma escala.

- Divisão Treino/Teste: Os dados foram divididos em 80% para treino e 20% para teste.

- Treinamento do Modelo: O modelo Random Forest foi treinado com 100 árvores (n_estimators=100).

- Avaliação do Modelo: Acurácia e um Relatório de Classificação foram usados para avaliar o desempenho.

E por fim, foi criado a função para predição, `predict_song_popularity`, em que ela faz predições para novas músicas, com base no ID.

In [ ]:

# Seleciona as features para o modelo
X = train_data[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                             'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 
                             'track_name_encoded', 'explicit', 'genre_encoded']]
y = train_data['popularity_target']

# Imputação de valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Padronização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Divide os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Cria e treinando o modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Faz as previsões no conjunto de teste
y_pred = rf.predict(X_test)

# Avalia o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

# Função para prever a popularidade de uma música
def predict_song_popularity(track_name_encoded):
    global imputer, scaler, rf

    # Seleciona as características da música a partir do ID codificado
    song_data = train_data[train_data['track_name_encoded'] == track_name_encoded].copy()

    # Se o DataFrame estiver vazio, retorna uma mensagem de erro
    if song_data.empty:
        return "Música não encontrada no conjunto de dados."

    # Realinha as colunas para que correspondam ao conjunto de dados original usado
    song_data = song_data.reindex(columns=X.columns, fill_value=0)

    # Imputação e padronização dos dados
    song_data_imputed = imputer.transform(song_data)
    song_data_scaled = scaler.transform(song_data_imputed)

    # Prevê a popularidade da música
    prediction = rf.predict(song_data_scaled)[0]

    if prediction == 1:
        result = "Popular"
    else:
        result = "Não Popular"

    return result

In [ ]:
# Exemplo de uso
track_name_id = 128  # ID codificado da música
print(predict_song_popularity(track_name_id))

## Importância das Features

Após treinar o modelo, podemos analisar a importância de cada uma das variáveis preditoras. O modelo de Random Forest identifica as principais features que influenciam diretamente a popularidade de uma música.

In [ ]:
# Verifica a importância das features do modelo treinado
importances = rf.feature_importances_

# Lista dos nomes das features
feature_names = [
'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                             'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 
                             'track_name_encoded', 'explicit', 'genre_encoded']

# Cria um df para organizar as importâncias das features
feature_importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Ordena as features pela importância
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibe as principais features
print("Principais Features:")
print(feature_importances.head(15))

# Visualiza as importâncias das features
plt.figure(figsize=(10, 6))
plt.title('Importância das Principais Features')
plt.barh(feature_importances['Feature'].head(15), feature_importances['Importance'].head(15))
plt.gca().invert_yaxis()  # Inverte o eixo y para que a feature mais importante apareça no topo
plt.xlabel('Importância')
plt.show()

## Tuning de Hiperparâmetros

Nesta parte, é aplicado um tuning com o hiperparâmetro `RandomizedSearchCV`, normalmente utilizado para encontrar as melhores combinações de parâmetros de um modelo de ML.

In [ ]:
# Define a grade de hiperparâmetros a serem testados
param_distributions = {
    'n_estimators': [200],
    'max_depth': [30],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True]
}

# Inicializa o modelo de Random Forest
rf = RandomForestClassifier(random_state=42)

# Aplica o RandomizedSearchCV com 5 folds (validação cruzada)
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions, 
                                   n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=1)

# Treina o modelo com Randomized Search
random_search.fit(X_train, y_train)

# Exibe os melhores hiperparâmetros encontrados
print("Melhores Hiperparâmetros:", random_search.best_params_)

# Avalia o modelo com os melhores hiperparâmetros
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Avalia o desempenho do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


## Validação no CSV de teste

Por fim, é feito treinamento para o modelo validar os valores no arquivo `test.csv`.

In [ ]:
# Seleciona as features e o alvo (target)
X = train_data[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 
                'track_name_encoded', 'explicit', 'genre_encoded']]
y = train_data['popularity_target']

# Imputação de valores faltantes e padronização dos dados de treino
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Divide os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Treina o modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Avalia o modelo com os dados de validação
y_pred_val = rf.predict(X_val)
print("Acurácia (validação):", accuracy_score(y_val, y_pred_val))
print("Relatório de Classificação (validação):\n", classification_report(y_val, y_pred_val))

# Seleciona as features do test_data (sem a variável alvo)
X_test_data = test_data[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                         'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
                         'track_name_encoded', 'explicit', 'genre_encoded']]

# Usa o mesmo imputador e escalador que foram treinados
X_test_imputed = imputer.transform(X_test_data)
X_test_scaled = scaler.transform(X_test_imputed)

# Faz as previsões no test_data
test_data['popularity_prediction'] = rf.predict(X_test_scaled)

# Exibe as primeiras linhas do DataFrame para confirmar que a coluna foi adicionada
print(test_data[['track_unique_id', 'popularity_prediction']].head())


## Criação do novo CSV

Aqui os resultados são salvos em um novo CSV

In [34]:
# Salva essas previsões em um outro arquivo CSV utilizando o `test.csv`
test_data[['track_unique_id', 'popularity_prediction']].to_csv('test_predictions_1.csv', index=False)